In [1]:
from ImportLeague import getLeague
from ImportData import getTeamData
from ImportData import getPlayerData
import pandas as pd
# from sklearn import linear_model
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('seaborn')
from tqdm import tqdm 
import warnings
warnings.filterwarnings("ignore")

In [2]:
# Import Data
myLeague = getLeague()
teamData_defense,teamData_offense = getTeamData()
playerData = getPlayerData(myLeague,teamData_defense,teamData_offense)

In [3]:
def get_active_roster(Roster):

    myRoster = pd.DataFrame(columns=['Position','Name','Proj Points'])

    # Create a dataframe of the full roster of players I own
    for player in Roster:
        playerDict = {}
        playerDict['Position'] = player.position
        playerDict['Name'] = player.name
        playerDict['Proj Points'] = player.stats[myLeague.nfl_week]['projected_points']
        myRoster = myRoster.append(playerDict, ignore_index = True)

    # Concatenate the n highest projected-points players with n being the allowed slots per position accross positions
    myRoster = pd.concat([myRoster[myRoster['Position'] == 'QB'].nlargest(1, ['Proj Points']),\
        myRoster[myRoster['Position'] == 'RB'].nlargest(2, ['Proj Points']),\
        myRoster[myRoster['Position'] == 'WR'].nlargest(3, ['Proj Points']),\
        myRoster[myRoster['Position'] == 'TE'].nlargest(1, ['Proj Points']),\
        myRoster[myRoster['Position'] == 'D/ST'].nlargest(1, ['Proj Points']),\
        myRoster[myRoster['Position'] == 'K'].nlargest(1, ['Proj Points'])])

    return myRoster


In [4]:
myRoster = get_active_roster(myLeague.teams[8].roster)
myRoster = myRoster.reset_index(drop=True)
myRoster

,Position,Name,Proj Points
0,QB,Dak Prescott,22.02
1,RB,Saquon Barkley,14.40
2,RB,Chase Edmonds,14.32
3,WR,Amari Cooper,12.07
4,WR,Marvin Jones Jr.,11.71
5,WR,Antonio Brown,11.46
6,TE,Noah Fant,8.62
7,D/ST,Broncos D/ST,7.18
8,K,Mason Crosby,8.08


In [5]:
# Find sizes of each free agency pool
sizes = []

for position in ['QB','RB','WR','TE','D/ST','K']:
    sizes.append(playerData[position][playerData[position]['Free Agency'] == True].shape[0])

In [7]:
def get_action(epsilon,Q_val,sizes_Dict,position_Dict):
    p_epsilon = np.random.uniform(0,1)
    if p_epsilon < epsilon:
        position = np.random.randint(9)
        free_agent = np.random.randint(sizes_Dict[position_Dict[position]])
        return (position,free_agent)

    action = np.unravel_index(Q_val.argmax(), Q_val.shape)
    while action[1] > (sizes_Dict[position_Dict[action[0]]] -1):
        Q_val[action[0],action[1]] = -100
        action = np.unravel_index(Q_val.argmax(), Q_val.shape)



    return action


In [35]:
sizes_Dict[position_Dict[2]]

5

In [38]:
Q_val = np.zeros([9,max(sizes)])
n_s = np.zeros([9,max(sizes)])
for slot in range(9):
    for null in range(sizes_Dict[position_Dict[slot]],max(sizes)):
        Q_val[slot][null] = -100

In [41]:
NUM_EPISODES = 1000
EPSILON = 0.3

# Sets Q-value table and vistis table to calculate inctemental averages
Q_val = np.zeros([9,max(sizes)])
n_s = np.zeros([9,max(sizes)])
sizes_Dict = dict(zip(['QB','RB','WR','TE','D/ST','K'], sizes))
position_Dict = dict(zip(range(9),['QB','RB','RB','WR','WR','WR','TE','D/ST','K']))
Baseline = myRoster['Proj Points'].sum()

for slot in range(9): 
    for null in range(sizes_Dict[position_Dict[slot]],max(sizes)): Q_val[slot][null] = -100


for episode in tqdm(range(NUM_EPISODES)):

    newRoster = myRoster.copy()

    action = get_action(EPSILON,Q_val,sizes_Dict,position_Dict)

    position = position_Dict[action[0]]
    newRoster['Name'][action[0]] = playerData[position][playerData[position]['Free Agency'] == True].iloc[action[1]]['Name']
    newRoster['Proj Points'][action[0]] = playerData[position][playerData[position]['Free Agency'] == True].iloc[action[1]]['Projected Points']

    reward = newRoster['Proj Points'].sum() - Baseline

    n_s[action[0],action[1]] += 1
    Q_val[action[0],action[1]] = Q_val[action[0],action[1]] + 1/n_s[action[0],action[1]] * (reward- Q_val[action[0],action[1]])


100%|██████████| 1000/1000 [00:01<00:00, 507.68it/s]


In [42]:
Q_val

array([[-4.720e+00, -7.210e+00, -2.390e+00, -7.470e+00, -3.090e+00,
        -6.720e+00, -8.800e+00, -6.540e+00],
       [-8.220e+00, -9.810e+00, -7.190e+00, -9.790e+00, -1.031e+01,
        -1.000e+02, -1.000e+02, -1.000e+02],
       [-8.140e+00, -9.730e+00, -7.110e+00, -9.710e+00, -1.023e+01,
        -1.000e+02, -1.000e+02, -1.000e+02],
       [-4.420e+00, -5.740e+00, -3.720e+00, -5.050e+00, -4.020e+00,
        -6.490e+00, -4.550e+00, -6.030e+00],
       [-4.060e+00, -5.380e+00, -3.360e+00, -4.690e+00, -3.660e+00,
        -6.130e+00, -4.190e+00, -5.670e+00],
       [-3.810e+00, -5.130e+00, -3.110e+00, -4.440e+00, -3.410e+00,
        -5.880e+00, -3.940e+00, -5.420e+00],
       [-2.110e+00, -3.170e+00, -5.810e+00, -3.630e+00, -1.000e+02,
        -1.000e+02, -1.000e+02, -1.000e+02],
       [-2.950e+00, -1.330e+00, -1.070e+00, -2.100e+00, -3.870e+00,
        -1.390e+00, -1.790e+00, -1.000e+02],
       [ 6.300e-01, -8.300e-01, -6.600e-01, -4.200e-01, -3.800e-01,
        -2.000e-02, -4.300e-

In [84]:
top_picks = np.unravel_index(np.argpartition(Q_val.flatten(), -4)[-4:], Q_val.shape)

recommends = pd.concat([playerData[position_Dict[top_picks[0][3]]][playerData[position_Dict[top_picks[0][3]]]['Free Agency'] == True].iloc[top_picks[1][3]],\
    playerData[position_Dict[top_picks[0][2]]][playerData[position_Dict[top_picks[0][2]]]['Free Agency'] == True].iloc[top_picks[1][2]],\
    playerData[position_Dict[top_picks[0][1]]][playerData[position_Dict[top_picks[0][1]]]['Free Agency'] == True].iloc[top_picks[1][1]],\
    playerData[position_Dict[top_picks[0][0]]][playerData[position_Dict[top_picks[0][0]]]['Free Agency'] == True].iloc[top_picks[1][0]]],axis=1).T

recommends = recommends[['Name','Projected Points']]
pos = []
for i in range(4): pos.append(position_Dict[top_picks[0][0]])
recommends['Postion'] =  np.flip(pos)
recommends['Q-Value'] = np.flip(Q_val[top_picks[0],top_picks[1]])

In [85]:
recommends

,Name,Projected Points,Postion,Q-Value
12,Harrison Butker,8.71,K,0.63
16,Rodrigo Blankenship,7.7,K,-0.38
17,Greg Joseph,8.06,K,-0.02
15,Jason Sanders,7.66,K,-0.42
